# ReACT Agents in Langgraph

- Standard Langgraph agents can be nodes and edges, where you can write basic nodes, conditional nodes, conditional edges and what not. 
- Writing agents that makes decisions is where you give away certain control of the agents flow to the llm, that decides what tools to call. 
- Certain models have capabilities to make tool calls. 
- The llms can make tool calls based on arguments defined in the arguments and description in the docstring. 

In [53]:
from typing import Annotated
from typing import Any, Optional , List
from typing_extensions import TypedDict
import os, json, logging
from neo4j import GraphDatabase

#Langgraph and Langchain dependencies. 
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
#from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.messages import ToolMessage
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables.graph_mermaid import draw_mermaid_png

#Manage Environment Variables. 
from dotenv import load_dotenv

load_dotenv()
auradb_conn=os.environ["AURADB_CONN"]
auradb_username = os.environ["AURADB_USERNAME"]
auradb_password = os.environ["AURADB_PASSWORD"]

In [54]:
import os , getpass
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()

def _set_env(var:  str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")





In [55]:
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    messages: Annotated[list, add_messages]


In [56]:
@tool("push_graph")
def push_graph(graph:str) -> Any:

    """ Push the json object with nodes and relationsips between entities to an AuraDB Graph database instance

    Args:
         graph: str -> Object serialized as a string with nodes and edges outlining relationships between the entities outlined in the passage 
                        given in process_passage tool. 
    
    Returns: 
         str -> Confirmation regarding whether or not the the graph push to auradb was successful.


    """

    AURA_CONNECTION_URI = auradb_conn
    AURA_USERNAME = "neo4j"
    AURA_PASSWORD = auradb_password

    # Driver instantiation
    driver = GraphDatabase.driver(
        AURA_CONNECTION_URI,
        auth=(AURA_USERNAME, AURA_PASSWORD)
    )

    import uuid

    input_data = json.loads(graph)["graph"] # Your provided JSON here
    
    # Transform nodes
    nodes = {item["node"]: {"id": str(uuid.uuid4())} for item in input_data}

    # Transform relationships
    relationships = []
    for item in input_data:
        source_node = item["node"]
        for rel in item["relationships"]:
            relationships.append({
                "source": source_node,
                "target": rel.get("target", rel.get("source")),
                "type": rel["type"].upper().replace(" ", "_")
            })


    def create_graph(tx):
        # Create nodes with unique IDs
        for name, props in nodes.items():
            nodes_result = tx.run("""
                MERGE (n:Species {name: $name})
                SET n.id = $id
            """, name=name, id=props["id"])

        # Create relationships
        for rel in relationships:
            relationships_result = tx.run("""
                MATCH (a:Species {name: $source}), (b:Species {name: $target})
                MERGE (a)-[r:RELATES_TO {type: $type}]->(b)
            """, source=rel["source"], target=rel["target"], type=rel["type"])
    
        return {"nodes": nodes_result.consume(), "relationships": relationships_result.consume()}

    
    try:
        with driver.session() as session:
            summary = session.execute_write(create_graph)
        
            return json.dumps({"messages":{"role":"assistant","content":f"Graph push successful with {summary['nodes'].counters} nodes and {summary['relationships'].counters}"}})
        
    except Exception as e:
        return json.dumps({"messages":[{"role":"assistant","content":f"Graph push failed with error: {str(e)}"}]})

In [57]:
@tool("process_passage")
def process_passage(text:str):
    
    """ 
     Identifies node and relationships between entities in the input text passage. 
     The objective of the function is to return a json object or Python dictionary, which will be passed
     to the push_graph function that stores it in the auradb graph databse. 

     args:
        text: str -> Text passage that describes Species and relationships that should be parsed by the llm and converted to json
      
     returns:
        str: Serialized Json string with a graph that has nodes and relationships outlined. this can be further processed and pushed to the 
              auradb database using push_graph
     
    
    """
    prompt = PromptTemplate.from_template("""For the following passage, give me ONLY a serialized json object that 
    describes the relationship between the species, compatible with a graph data structure. 
    For example a graph relationship: Lion --> (predator) --> Antelope should output a serialized json string \n

    the passage starts from the next line \n {text} \n


    *** Requirements ***

        - Only return the json object. Nothing else.
        - each node relationship should have only one source or target. Not a list. 
        - Add any node and relationship properties (ecosystem, etc.) based on the passage

    *** Example output for a relationship ***

    {{
       "node": "Jaguar",
       "relationships": [
         {{
           "type": "predator",
           "target": "Capybara"
         }}
       ]
     }}

    """)
    
    parser_llm = ChatOllama(model="mistral:instruct")

    output_schema = { "type": "array",
                      "items": { "type": "object",
                           "properties": 
                           { "node": { "type": "string" },
                                       "relationships": { "type": "array",
                                       "items": { "type": "object",
                                                  "properties": { "type": { "type": "string" },
                                                  "target": { "type": "string" } },
                                                  "required": ["type", "target"] } },
                                                  "ecosystem": { "type": "string" } },
                                                "required": ["node", "relationships", "ecosystem"] } }
    
    structured_llm = parser_llm.with_structured_output(output_schema)

    formatted_prompt = prompt.format(text=text)
    response = structured_llm.invoke(formatted_prompt)
    parser = JsonOutputParser()
    print(response)
    return json.dumps({"graph": parser.parse(response)})
    

In [58]:
llm = ChatOllama(model="mistral:instruct")
tools = [process_passage, push_graph]
llm_with_tools = llm.bind_tools(tools)

In [59]:
from langchain_core.runnables import RunnableConfig

def agent(
        state:State, 
        config:RunnableConfig
        ):
    system_prompt = SystemMessage(
        """You are a helpful Research Assistant to answer queries about Species in Ecosystems. Use tools if necessary."""
    )
    response = llm_with_tools.invoke([system_prompt] + state["messages"], config)
    return {"messages": [response]}

In [60]:
tools_by_name = {tool.name: tool for tool in tools}
def call_tool(state:State):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content = json.dumps(tool_result),
                name = tool_call["name"],
                tool_call_id =  tool_call["id"]
            )
        )
    return {"messages": outputs}

In [61]:
def router(state:State):

    last_message = state["messages"][-1]

    if last_message.tool_calls:
        return "tools"
    
    else:
        return "end"

In [62]:
from langgraph.graph import StateGraph , END

workflow = StateGraph(State)
workflow.add_node("agent", agent)
workflow.add_node("tools", call_tool)   
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    router,
    {"tools":"tools", "end":END})

workflow.add_edge("tools","agent")

agent = workflow.compile()

In [63]:
from IPython.display import Image, display
import mermaid as md 
from mermaid.graph import Graph

mermaid_code = agent.get_graph().draw_mermaid()
render = md.Mermaid(mermaid_code)

render


In [64]:
passage = """
In the tropical rainforest ecosystem, several species exhibit a variety of biological relationships that illustrate evolutionary, ecological, and taxonomic connections. The Jaguar (Panthera onca) is a top predator and preys upon the Capybara (Hydrochoerus hydrochaeris) and the Green Anaconda (Eunectes murinus). Both the Capybara and Green Anaconda share the same wetland habitat, demonstrating ecological co-occurrence.
The Green Anaconda often competes with the Harpy Eagle (Harpia harpyja) for prey such as the Howler Monkey (Alouatta palliata). The Howler Monkey, an arboreal primate, is closely related to the Spider Monkey (Ateles geoffroyi), with both belonging to the family Atelidae. This taxonomic relationship indicates a common evolutionary ancestor.
The Harpy Eagle also preys on the Sloth (Bradypus variegatus), which has a mutualistic relationship with several species of algae, such as Trentepohlia spp., that grow on its fur, providing camouflage. Additionally, the Sloth shares a symbiotic relationship with the Moth (Cryptoses choloepi), which uses the sloth’s fur for habitat.
From an evolutionary standpoint, the Capybara and the Jaguar belong to different orders: Rodentia and Carnivora, respectively, reflecting deep phylogenetic divergence. The Sloth is taxonomically grouped within the order Pilosa, distinct from the other mammals mentioned.
Together, these species form a complex web of predation, competition, symbiosis, and shared evolutionary history, making the tropical rainforest a dynamic and interconnected ecosystem.
"""

In [65]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", f"Process the following passage, identify the relationships between species and push it to the auradb graph database, {passage}")]}
print_stream(agent.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Process the following passage, identify the relationships between species and push it to the auradb graph database, 
In the tropical rainforest ecosystem, several species exhibit a variety of biological relationships that illustrate evolutionary, ecological, and taxonomic connections. The Jaguar (Panthera onca) is a top predator and preys upon the Capybara (Hydrochoerus hydrochaeris) and the Green Anaconda (Eunectes murinus). Both the Capybara and Green Anaconda share the same wetland habitat, demonstrating ecological co-occurrence.
The Green Anaconda often competes with the Harpy Eagle (Harpia harpyja) for prey such as the Howler Monkey (Alouatta palliata). The Howler Monkey, an arboreal primate, is closely related to the Spider Monkey (Ateles geoffroyi), with both belonging to the family Atelidae. This taxonomic relationship indicates a common evolutionary ancestor.
The Harpy Eagle also preys on the Slo

ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value=[{'node': 'Jaguar', 'rela... 'tropical rainforest'}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type